# Example notebook: cross-match data with SkyQuery

SciServer Compute can talk to other components of SciServer through a series of <em>modules</em>, one for each component. This example notebook shows how to use the <strong><code>SciServer.SkyQuery</code></strong> module to perform sophisticated cross-matches among astronomical datasets.

You are welcome (encouraged!) to copy these examples into another folder and modify them to meet your needs. You can use them as a starting point to create your own scripts. Please do not edit this notebook directly, because your edits may be overwritten if changes to the SciServer modules require changes to these example notebooks.

To run the example Python scripts in this notebook, click in any of the Code cells below (the ones with the gray backgrounds). Click the play button at the top of the window (just below the menubar) to run the script, or press Shift-Enter. The output of each cell's script will appear directly below the cell.

## Import modules

Like any Python modules, the SciServer modules must be imported before being used. The next code block first imports the SciServer modules you will need for this example notebook, then imports some other required modules. Comments in the code block explain what each module does. To learn how to import other modules, see the Python 3.5 import documentation (https://docs.python.org/3.5/reference/import.html), or the documentation of the module(s) you are trying to import.

In [ ]:
from SciServer import SkyQuery
print('Imported SciServer modules')     # Work with SkyQuery through Compute

from pprint import pprint
print('Imported other needed modules')  # print data structures in readable format (https://docs.python.org/3.5/library/pprint.html)

## List available datasets

The function <strong><code>SkyQuery.listAllDatasets()</code></strong> (takes no parameters) lists all the datasets available to you through SkyQuery.

The function returns a list of dictionaries, one for each available dataset. You can cross-match among any of the datasets by using its <code>name</code>.

In [ ]:
datasets = SkyQuery.listAllDatasets()
pprint(datasets)

## List the contents of your MyDB

The function <strong><code>SkyQuery.getDatasetInfo(datasetName)</code></strong> returns the contents of a specified database. You can use it to show the contents of any of the databases you found in the last code cell; here, we run in with <code>MyDB</code> to show the contents of your MyDB.

The function returns a list of dictionaries, one for table in the specified dataset. Each table has a <strong><code>name</code></strong> by which it can be referred.

In [ ]:
info = SkyQuery.getDatasetInfo("MyDB")
print(info)
print('\n')

tables = SkyQuery.listDatasetTables("MyDB")
pprint(tables)

## Manage jobs

Like CasJobs, SkyQuery manages its requests (both queries and cross-match requests) as <strong>jobs</strong>. You can submit the request using the <strong><code>SkyQuery.SubmitJob(query, queue)</code></strong> function. 

When you submit a query or a cross-identification request to SkyQuery, it creates a job and assigns a <em>jobID</em> that you can use to check on the status of the query. 

In [ ]:
# Define a very simple query
SkyQuery_Query = "select 4.5 as Column1, 5.5 as Column2"

# Submit that simple query as a job to SkyQuery
jobId = SkyQuery.submitJob(query=SkyQuery_Query, queue="quick")
print('submitted jobId',jobId)

In [ ]:
#get the status of a submitted job using jobDescription(jobId) 

secondjobID = SkyQuery.submitJob(query=SkyQuery_Query, queue="quick")

jobDescription = SkyQuery.getJobStatus(jobId=secondjobID)
pprint(jobDescription)

In [ ]:
# Submit a job, wait for it to finish, and then get its status
#thirdjobID = SkyQuery.submitJob(query=SkyQuery_Query, queue="quick")
##jobDescription = SkyQuery.waitForJob(jobId=thirdjobID, verbose=True)

#print('\n')
#print("job description:")

In [ ]:
# Cancel a job that is running, and then get its status to prove it is has been cancelled

fourthjobID = SkyQuery.submitJob(query=SkyQuery_Query, queue="long")
SkyQuery.cancelJob(fourthjobID)

print("job status:")
pprint(SkyQuery.getJobStatus(jobId=fourthjobID))

### Manage jobs: queues

To be fair to all users, SkyQuery uses a system of <strong>queues</strong> to balance its load. That means that queries that finish quickly will get priority over longer queries to minimize wait time &mdash; but that those longer queries will run in the background and will always complete eventually.

You can use functions in the SkyQuery module to access information about what queues are available, and how you have interacted with those queues.

In [ ]:
# Return a list of all available queues in SkyQuery
SkyQuery.listQueues()

In [ ]:
# Get a list of all the jobs you have run in SkyQuery.
#  This code cell retruns two dictionaries: one for jobs submitted through the Quick queue...
#   and one for jobs submitted through for the Long queue.

quickJobsList = SkyQuery.listJobs('quick')
longJobsList = SkyQuery.listJobs('long')

print('Quick Jobs:')
pprint(quickJobsList)

print('\n')
print('Long jobs:')
pprint(longJobsList)

## Upload data to your MyDB

One of the most important features of SkyQuery is its ability to cross-match your data with a variety of online big data astronomical catalogs. But to perform such a cross-match, you must first upload your data to your MyDB. This section shows you how to use functions in the SkyQuery module to do that.

In [ ]:
#define a very simple CSV file containing data to upload to your MyDB in SkyQuery

SkyQuery_TestTableName = "TestTable_SkyQuery"
SkyQuery_TestTableCSV = u"Column1,Column2\n15.5,16.5\n"

print('Simple CSV file created')

In [ ]:
# Upload your sample CSV file using the function SkyQuery.uploadTable(uploadData, TableName, ...). 
## The function returns True if the table has been properly uploaded.
### NOTE: this works ONLY if the table you specified does not already exist.

result = SkyQuery.uploadTable(uploadData=SkyQuery_TestTableCSV, tableName=SkyQuery_TestTableName, datasetName="MyDB", format="csv")
print(result)

In [ ]:
# You can also download data from your MyDB using SkyQuery.getTable(tableName, ...)
## This command returns a Pandas dataframe.

table = SkyQuery.getTable(tableName=SkyQuery_TestTableName, datasetName="MyDB")
table

## Manage MyDB

You can also use various SkyQuery module functions to manage your MyDB. The following code cells give some examples.

In [ ]:
#list tables inside dataset

tables = SkyQuery.listDatasetTables("MyDB")
pprint(tables)

In [ ]:
#get info for a specified table:

info = SkyQuery.getTableInfo(tableName="webuser." + SkyQuery_TestTableName, datasetName="MyDB")
pprint(info)

In [ ]:
#get dataset table columns info

columns = SkyQuery.listTableColumns(tableName="webuser." + SkyQuery_TestTableName, datasetName="MyDB")
pprint(columns)

In [ ]:
#drop (i.e. delete) table from dataset.

result = SkyQuery.dropTable(tableName=SkyQuery_TestTableName, datasetName="MyDB");
print(result)

## Thank you!

Thanks for reviewing this SciServer example notebook. You can use this notebook as a template to develop your own notebooks, but please do so in a copy rather than in the original example notebook.

As you begin to use any of our SciServer modules in your own notebooks, consult the SciServer scripting documentation at http://www.sciserver.org/docs/sciscript-python/SciServer.html.

If you have questions, please email the SciServer helpdesk at sciserver-helpdesk@jhu.edu .